In [ ]:
import os

# Cartelle
input_folder = "./JSON/no vincoli"
output_folder = "./JSON/no vincoli_cleaned"

os.makedirs(output_folder, exist_ok=True)


def is_empty(line):
    return line.strip() == ""


for filename in os.listdir(input_folder):

    input_path = os.path.join(input_folder, filename)

    if not os.path.isfile(input_path):
        continue

    with open(input_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    cleaned = []

    i = 0
    n = len(lines)

    while i < n:

        # Controlla se inizia una sequenza vuota-testo-vuota
        if (
            is_empty(lines[i]) and
            i + 2 < n and
            not is_empty(lines[i + 1]) and
            is_empty(lines[i + 2])
        ):

            # Salta tutto il blocco
            i += 1

            while i < n:

                # Salta una riga di testo
                if i < n and not is_empty(lines[i]):
                    i += 1

                # Salta una riga vuota
                if i < n and is_empty(lines[i]):
                    i += 1
                else:
                    break

        else:
            cleaned.append(lines[i])
            i += 1

    output_path = os.path.join(output_folder, filename)

    with open(output_path, "w", encoding="utf-8") as f:
        f.writelines(cleaned)

    print(f"Processato: {filename}")

print("Tutti i file completati.")


Processato: solutionCLEX1.json
Processato: solutionCLEX2.json
Processato: solutionCLEX3.json
Processato: solutionCLEX4.json
Processato: solutionCLEX5.json
Processato: solutionCLEX6.json
Processato: solutionHeu1.json
Processato: solutionHeu2.json
Processato: solutionHeu3.json
Processato: solutionHeu4.json
Processato: solutionHeu5.json
Processato: solutionHeu6.json
Tutti i file completati.


In [8]:
import json
import math

def mean(values):
    return sum(values) / len(values) if values else 0.0

def std_dev(values):
    filtered = [v for v in values if v != 0]
    if len(filtered) <= 1:
        return 0.0
    m = mean(filtered)
    variance = sum((x - m) ** 2 for x in filtered) / len(filtered)
    return math.sqrt(variance)


def analyze_solution(solution):
    sprint_risks = []
    sprint_uncertainties = []
    sprint_capacity_usage = []

    sprint_utilities = []   # utilità per sprint
    total_utility = 0       # utilità totale

    for sprint in solution.get("sprints", []):
        stories = sprint.get("stories", [])
        if not stories:
            continue

        total_risk = sum(story.get("risk", 0) for story in stories)
        total_uncertainty = sum(story.get("uncertainty", 0) for story in stories)

        # utilità dello sprint
        sprint_utility = sum(story.get("utility", 0) for story in stories)

        capacity = sprint.get("capacity", 0)
        used_capacity = sprint.get("used_capacity", 0)

        if capacity > 0:
            sprint_capacity_usage.append(used_capacity / capacity)

        sprint_risks.append(total_risk)
        sprint_uncertainties.append(total_uncertainty)

        sprint_utilities.append(sprint_utility)
        total_utility += sprint_utility


    # Calcolo metà utilità
    half_utility = total_utility / 2

    cumulative = 0
    sprint_to_half_utility = None

    for i, u in enumerate(sprint_utilities, start=1):
        cumulative += u
        if cumulative >= half_utility:
            sprint_to_half_utility = i
            break


    return {
        "solution_name": solution.get("name", "unknown"),
        "num_sprint_considered": len(sprint_risks),

        # Rischio
        "risk_std_dev": std_dev(sprint_risks),
        "risk_max": max(sprint_risks) if sprint_risks else 0.0,
        "risk_min": min(sprint_risks) if sprint_risks else 0.0,

        # Incertezza
        "uncertainty_std_dev": std_dev(sprint_uncertainties),
        "uncertainty_max": max(sprint_uncertainties) if sprint_uncertainties else 0.0,
        "uncertainty_min": min(sprint_uncertainties) if sprint_uncertainties else 0.0,

        # Capacità
        "avg_capacity_usage": mean(sprint_capacity_usage),

        "sprint_to_half_utility": sprint_to_half_utility
    }


In [9]:
import glob
import json
import os


json_files = glob.glob("./JSON/standard_cleaned/*.json")

print("=== GENERAZIONE RESOCONTI ===\n")


for file in json_files:

    # Nome file senza estensione
    base_name = os.path.splitext(file)[0]

    # Nome output personalizzato
    output_file = f"{base_name}_resoconto.txt"

    print(f"Elaboro: {file} → {output_file}")

    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)

    with open(output_file, "w", encoding="utf-8") as f_out:

        f_out.write("=== RESOCONTO PER SOLUZIONE ===\n\n")

        for solution in data.get("solutions", []):

            r = analyze_solution(solution)

            line = (
                f" {r['solution_name']}  &  "
                f" {r['num_sprint_considered']}  &  "
                f" {r['risk_std_dev']:.1f}  &  "
                f" {r['risk_max']:.1f}  &  {r['risk_min']:.1f}  &  "
                f" {r['uncertainty_std_dev']:.1f}  &  "
                f" {r['uncertainty_max']:.1f}  &  {r['uncertainty_min']:.1f}  &  "
                f" {r['avg_capacity_usage']:.2%}  &  "
                f" {r['sprint_to_half_utility']}  \\\\"
            )

            print(line)
            f_out.write(line + "\n")


print("\nTutti i resoconti sono stati creati.")


=== GENERAZIONE RESOCONTI ===

Elaboro: ./JSON/standard_cleaned\solutionCLEX1.json → ./JSON/standard_cleaned\solutionCLEX1_resoconto.txt
 stance_06_01.txt  &   5  &   2.2  &   10.6  &  5.1  &   2.1  &   10.1  &  4.3  &   95.59%  &   2  \\
 stance_06_02.txt  &   7  &   2.3  &   9.2  &  1.8  &   2.0  &   7.7  &  1.6  &   84.68%  &   2  \\
 stance_06_03.txt  &   6  &   3.6  &   13.5  &  1.9  &   4.3  &   15.7  &  1.7  &   88.89%  &   2  \\
 stance_06_04.txt  &   5  &   3.1  &   11.2  &  3.0  &   3.0  &   11.1  &  3.4  &   86.65%  &   2  \\
 stance_06_05.txt  &   5  &   2.0  &   10.1  &  4.0  &   2.1  &   9.8  &  3.8  &   98.34%  &   2  \\
   &   23  &   1.1  &   6.4  &  2.5  &   0.9  &   5.9  &  2.7  &   95.94%  &   9  \\
   &   21  &   1.5  &   7.6  &  1.6  &   1.1  &   6.1  &  1.2  &   93.09%  &   7  \\
   &   21  &   1.5  &   6.5  &  1.2  &   1.1  &   5.8  &  1.7  &   95.87%  &   8  \\
   &   23  &   1.3  &   7.8  &  2.4  &   1.1  &   7.5  &  2.5  &   96.12%  &   9  \\
   &   21  &   1